# pip install openai==0.28 설치하셔야 됩니다!!

In [418]:
import gradio as gr
import openai
import base64
import time
import mimetypes
import fitz
import spacy
import faiss
import numpy as np


In [419]:
from sentence_transformers import SentenceTransformer

In [420]:
nlp = spacy.load("en_core_web_sm")

In [465]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [460]:
openai.api_base = "https://ed48-34-125-52-90.ngrok-free.app/v1"  # 또는 ngrok URL
openai.api_key = "EMPTY"

In [461]:
try:
    models = openai.Model.list()
    print(models)
except Exception as e:
    print(f"Error: {e}")

{
  "object": "list",
  "data": [
    {
      "id": "llava-hf/llava-1.5-7b-hf",
      "object": "model",
      "created": 1733090640,
      "owned_by": "vllm",
      "root": "llava-hf/llava-1.5-7b-hf",
      "parent": null,
      "max_model_len": 4096,
      "permission": [
        {
          "id": "modelperm-daee2b0296d748749df3787ea1692236",
          "object": "model_permission",
          "created": 1733090640,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ]
    }
  ]
}


In [466]:
model_vllm = models.data[0].id


In [463]:
def encode_base64_content_from_file(image_path: str) -> str:
    """Encode a content retrieved from image_path to base64 format."""
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

In [464]:
def remove_image_url_from_history(history):
    for msg in history:
        # content가 리스트인 경우에만 처리
        if isinstance(msg.get("content"), list):
            # "type": "image_url"이 아닌 항목만 남김
            msg["content"] = [c for c in msg["content"] if c.get("type") != "image_url"]
    return history

In [427]:
def make_openai_msg(message,history):
  content = []
  print("message[text]", message["text"])
  if message["files"]:
    history = remove_image_url_from_history(history)
    for x in message["files"]:
      print(x)
      image_url = f"data:image/jpeg;base64,{encode_base64_content_from_file(x)}"
      content.append({"type": "image_url",
                      "image_url":{"url": image_url}})
    if message["text"]:
      content.append({"type":"text",
                      "text": message["text"]})
    else:
      content.append({"type":"text",
                      "text":"explain this picture(or image)"})
    new_history = history + [{"role":"user",
          "content":content}]
    print(new_history)
    return gr.update(value=new_history)
  else:
    new_history = history + [{"role": "user",
                "content": message["text"]}]
    print(new_history)
    return gr.update(value=new_history)
  

In [428]:
def add_message(history, message):
    for x in message["files"]:
        history.append({"role": "user", "content": {"path": x}})
    if message["text"] is not None:
        history.append({"role": "user", "content": message["text"]})
    else: history.append({"role": "user", "content": "explain this image or picture"})
    return history, gr.MultimodalTextbox(value=None, interactive=False)

In [471]:
def bot(history: list,msg_history):
    
    chat_completion = openai.ChatCompletion.create(
        model=model_vllm,  # Replace with your model ID
        messages=msg_history,
        max_tokens=128,  # Configurable for more detailed responses
    )
    result = chat_completion.choices[0].message["content"]
    print("bot생성 답변: ",result)
    history.append({"role": "assistant", "content": ""})
    update_history = msg_history + [{"role": "assistant", "content":result}]
    print(update_history)
    for character in result:
        history[-1]["content"] += character
        time.sleep(0.05)
        yield history, gr.update(value=update_history)

In [430]:
def bot2(history: list,msg_history):
    
    result = "이잉 기모링~"
    history.append({"role": "assistant", "content": ""})
    update_history = msg_history + [{"role": "assistant", "content":result}]
    print(update_history)
    for character in result:
        history[-1]["content"] += character
        time.sleep(0.05)
        yield history, gr.update(value=update_history)

In [431]:
def extract_text_from_pdf(pdf_path):
    # PDF 파일 열기
    doc = fitz.open(pdf_path)

    # 모든 페이지에서 텍스트 추출
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # 페이지 로드
        text += page.get_text()  # 텍스트 추출

    return text

In [432]:
def make_chunk(text):
  chunk_list = []
  doc = nlp(text)
  for sent in (doc.sents):
    chunk = sent.text
    chunk_list.append(chunk)
  return chunk_list

In [433]:
def embedding_text(chunk_list):
  embedding_list =[]
  for chunk in chunk_list:
    embedding = model.encode(chunk)
    embedding_list.append(np.array(embedding))
  embedding_tensor = np.array(embedding_list,dtype=np.float32)
  return embedding_tensor

In [434]:
def load_to_faiss(embedding_list):
  dimension = embedding_list.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embedding_list)
  return index

In [468]:
def process_pdf(message):
    try:
        if message["files"]:
            new_message={
                "files":[],
                "text":""
            }
            text_from_pdf = None
            for x in message["files"]:
                mime_type, _ = mimetypes.guess_type(x)
                if mime_type == "application/pdf":
                    print("This is a PDF file")
                    text_from_pdf = extract_text_from_pdf(x)
                    chunk_list = make_chunk(text_from_pdf)
                    print("chunk_list: ",chunk_list)
                    embedding_list = embedding_text(chunk_list)
                    print("embedding_list: ",embedding_list)
                    faiss_index = load_to_faiss(embedding_list)
                    print("faiss완료!: ",faiss_index.ntotal)
                    query = model.encode([message["text"]])
                    print("query임베딩 완료!")
                    query_np = np.array(query).astype("float32")
                    print("query ndarray변환 완료!")
                    _,indices= faiss_index.search(query_np, 5)
                    print("result: ",indices)
                    print("indices type: ", type(indices[0][0]))
                    text = ""
                    for idx in indices[0]:
                        text += "Reference sentence: " + chunk_list[idx]
                        text = str(text)
                    new_message["text"] = (
                        "Please answer the question by referring to the given reference sentence."
                        + text
                        + " Question: "
                        + message["text"]
                    )
        return gr.update(value=new_message)
    except Exception as e:
        print(f"Error in process_pdf: {e}")
        return gr.update(value=new_message)


In [472]:
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(elem_id = "chatbot", bubble_full_width=False, type="messages")

  chat_input = gr.MultimodalTextbox(
    interactive=True,
    file_count="multiple",
    placeholder="메세지나 파일을 입력하세요",
    show_label=False,
  )
  json_output = gr.JSON(visible=False)
  msg_history = gr.JSON(visible=False,value=[{"role": "system", "content":"your are intelligent chatbot."}])

  
  pdf_check = chat_input.submit(
    process_pdf, chat_input, json_output
  )
  

  user_msg = pdf_check.then(
    make_openai_msg, [json_output,msg_history], msg_history
  )
  
  chat_msg = user_msg.then(
    add_message, [chatbot, chat_input], [chatbot,chat_input]
  )

  bot_msg = chat_msg.then(bot, [chatbot,msg_history], [chatbot,msg_history], api_name="bot_response")
  bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

In [473]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


This is a PDF file
chunk_list:  ['This is a simple test text to check the functionality of PDF text extraction tools.', 'The \npurpose of this test is to verify if the text is displayed properly when the PDF is opened. \n', 'It is important to ensure that the text can be extracted easily and without any distortion. \n', 'Testing PDF text extraction can help identify issues such as encoding problems or \nimproper rendering of fonts.', 'The text includes several lines, which will help assess the \nalignment and formatting of the content. \n', 'This test text includes both short and long sentences, which can be useful in evaluating \nthe extraction process.', 'By testing various kinds of text, you can get a better \nunderstanding of how well a PDF tool handles different formats.', 'The final result should \nshow that all characters and sentences appear correctly when extracted.', 'It’s also helpful \nto test PDFs in different languages to ensure that the tool handles multiple character \n